In [1]:
# Set your parameters
class_type = "GE"  # GE or CM

In [2]:
import numpy as np
import pandas as pd
df = pd.read_excel(f'{class_type}.xlsx')

In [3]:
course_list = df.iloc[:, 2].tolist()

In [4]:
course_list = course_list[3:]

In [5]:
course_list = list(set(course_list))

In [6]:
with open(f'{class_type} course name.txt', 'w') as file:
    for course in course_list:
        file.write(str(course) + '\n')

In [7]:
course_list[:10]

['GEST1007',
 'GEGA3001',
 'GESB1009',
 'GELH2005',
 'GEST1002',
 'GEGA1008',
 'GESB2002',
 'GEST2003',
 'GEST1017',
 'GELH2010']

In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
from tqdm import tqdm

courses = []
with open(f'{class_type} course name.txt', 'r') as file:
    lines = file.read().splitlines()
    courses = [line.strip() for line in lines]

df = pd.DataFrame(columns=['course_name', 'name', 'overall_grade', 'grade', 'difficulty', 'useful', 'comments'])

wd = webdriver.Edge()
wd.implicitly_wait(2)

total_entries = 0

pbar = tqdm(total=len(courses), desc="Processing courses", ncols=100)

for course_name in courses:
    wd.get(f"https://www.umeh.top/course/{course_name}")
    time.sleep(1)
    
    try:
        teacher_blocks = WebDriverWait(wd, 2).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'a[href^="/reviews/"]'))
        )
        
        course_entries = 0
        for block in teacher_blocks:
            try:
                name = block.find_element(By.CSS_SELECTOR, 'div.break-words').text
                overall_grade_element = WebDriverWait(block, 2).until(
                    EC.presence_of_element_located((By.XPATH, './/div[@class="text-sm font-semibold"]/div[2]'))
                )
                overall_grade = overall_grade_element.text
                
                stats = block.find_elements(By.CSS_SELECTOR, 'div.flex.flex-row.text-xs.font-semibold.space-x-2 > div')
                grade = stats[0].find_element(By.CSS_SELECTOR, 'div:nth-child(2)').text
                difficulty = stats[1].find_element(By.CSS_SELECTOR, 'div:nth-child(2)').text
                useful = stats[2].find_element(By.CSS_SELECTOR, 'div:nth-child(2)').text
                comments = int(stats[3].find_element(By.CSS_SELECTOR, 'div:nth-child(2)').text)
                
                new_row = pd.Series([course_name, name, overall_grade, grade, difficulty, useful, comments], index=df.columns)
                df.loc[len(df)] = new_row
                
                total_entries += 1
                course_entries += 1
            
            except Exception as e:
                pbar.write(f"something went wrong with {course_name}")
                continue
        
            pbar.set_description(f"Processing {course_name}: {name} | Total entries: {total_entries}")
    
    except Exception as e:
        pbar.write(f"something went wrong with {course_name}")
        continue
    
    pbar.update(1)

wd.quit()
pbar.close()

Processing GEST1002: CHEN QIANG / ZHENG WENHUA | Total entries: 61:   9%| | 5/58 [00:15<02:19,  2.63

something went wrong with GEGA1008


Processing GEST1017: WANG RUIBING | Total entries: 79:  14%|█▏       | 8/58 [00:25<02:16,  2.72s/it]

something went wrong with GELH2010


Processing GELH2002: GLEN TIMMERMANS | Total entries: 80:  16%|▉     | 9/58 [00:30<02:55,  3.57s/it]

...

Processing GEST1015: HUI KWUN NAM | Total entries: 276:  74%|█████▏ | 43/58 [01:45<00:26,  1.75s/it]

something went wrong with GELH1009


Processing GEST2002: SIT HO YI | Total entries: 341:  86%|████████▌ | 50/58 [02:04<00:16,  2.02s/it]

something went wrong with GEGA2008


Processing GESB2001: CHEN FEIER | Total entries: 343:  88%|███████▉ | 51/58 [02:08<00:17,  2.51s/it]


In [9]:
# save to csv
df.to_csv(f'{class_type}_raw.csv', index=False)

In [11]:
df[:10]

,course_name,name,overall_grade,grade,difficulty,useful,comments
0,GEST1007,GARY WONG / LI GANG / QIAN LUO / WONG KOON HO,N/A,N/A,N/A,N/A,0
1,GEST1007,GARY WONG / WONG KOON HO,C+,B-,C+,B-,81
2,GEST1007,GARY WONG,N/A,N/A,N/A,N/A,0
3,GEST1007,LI GANG,N/A,N/A,N/A,N/A,0
4,GEST1007,GARY WONG / HONGJIE ZHANG / LI GANG / QIAN LUO...,N/A,N/A,N/A,N/A,0
5,GEST1007,WONG KOON HO,N/A,N/A,N/A,N/A,0
6,GEST1007,QIAN LUO,N/A,N/A,N/A,N/A,0
7,GEST1007,HONGJIE ZHANG,N/A,N/A,N/A,N/A,0
8,GEGA3001,WANG JIANWEI,C+,C,C+,C+,25
9,GEGA3001,CHUA YEE HONG,D,D+,D,D,54
